In [1]:
!pip install langchain qdrant-client -q
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.6/306.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 16.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.3 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 2.9 MB/s eta 0:00:00


In [2]:
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient

# **Tải về văn bản Chương trình đào tạo 2023**

In [3]:
!pip install gdown

In [4]:
import gdown

# Link tải file ZIP
# https://drive.google.com/file/d/12U_2V2y5bUB9JLx9DdqDFmGH_OQQLW3o/view?usp=sharing
zip_file_id = '12U_2V2y5bUB9JLx9DdqDFmGH_OQQLW3o'
gdown.download(f'https://drive.google.com/uc?id={zip_file_id}', 'Data.zip', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=12U_2V2y5bUB9JLx9DdqDFmGH_OQQLW3o
To: /content/Data.zip
100%|██████████| 163k/163k [00:00<00:00, 67.2MB/s]


'Data.zip'

In [5]:
import zipfile

with zipfile.ZipFile('Data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

# **Xử lý dữ liệu**

In [6]:
folder_path = "/content/Data"
documents = []
for file_name in os.listdir(folder_path):
    if file_name.endswith(".txt"):
        file_path = os.path.join(folder_path, file_name)
        loader = TextLoader(file_path)
        documents.extend(loader.load())  # Nạp tài liệu từ từng file

In [7]:
# Kiểm tra số lượng và kiểu dữ liệu của documents
print(f"Số lượng tài liệu: {len(documents)}, Kiểu dữ liệu của documents: {type(documents)}")

# Kiểm tra nội dung và metadata của tài liệu đầu tiên
print(f"Nội dung của tài liệu đầu tiên: {documents[0].page_content}")
print(f"Metadata của tài liệu đầu tiên: {documents[0].metadata}")
print(f"Source trong metadata của tài liệu đầu tiên: {documents[0].metadata.get('source', 'Không có thông tin về source')}")

Số lượng tài liệu: 10, Kiểu dữ liệu của documents: <class 'list'>
Nội dung của tài liệu đầu tiên: Ngành Trí tuệ nhân tạo
Chương trình: Đại học Chính quy
Khóa tuyển: 2023
(Ban hành kèm theo Quyết định số 1712/QĐ - KHTN ngày 07/9/2023 
của Hiệu trưởng Trường Đại học Khoa học Tự nhiên, ĐHQG-HCM) 

1. Thông tin chung về chương trình đào tạo
1.1. Tên ngành: 
- Tiếng Việt: Trí tuệ nhân tạo
- Tiếng Anh: Artificial Intelligence
1.2. Mã ngành: 7480107
1.3. Trình độ đào tạo : Đại học
1.4. Tên chương trình : Cử nhân Trí tuệ nhân tạo 
1.5. Loại hình đào tạo : Chính quy 
1.6. Thời gian đào tạo : 4 năm
1.7. Tên văn bằng sau khi tốt nghiệp: 
- Tiếng Việt : Cử nhân Trí tuệ nhân tạo 
- Tiếng Anh Bachelor of Science in Artificial Intelligence 
1.8. Ngôn ngữ giảng dạy tiếng Việt
1.9. Nơi đào tạo: 
- Cơ sở 1: 227 Nguyễn Văn Cừ, P4, Q5, Tp. HCM  
- Cơ sở 2: Phường Linh Trung, Thành phố Thủ Đức, Tp. HCM 

2. Mục tiêu đào tạo
2.1. Mục tiêu chung
Mục tiêu của chương trình là nhằm đào tạo ra các sinh viên tốt 

#**Khai báo biến môi trường**
- Lưu ý: Nếu API hết hạn truy cập chương trình sẽ không hoạt động và báo lỗi, cần thay API key mới


In [ ]:
# Cài đặt thông tin truy cập vector database
QDRANT_API_KEY = ""
QDRANT_URL = ""
EMBEDDINGS_MODEL_NAME="intfloat/multilingual-e5-base"
# EMBEDDINGS_MODEL_NAME="sentence-transformers/LaBSE"
HUGGINGFACE_API_KEY= ""

#**Sử dụng RecursiveCharacterTextSplitter của LangChain**

In [9]:
COLLECTION_NAME = "ITUS_e5_R_600"

In [10]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Giả sử documents là một danh sách các document, và mỗi document có tên file trong metadata.
# Cập nhật metadata để chứa chỉ tên file

text_splitter = RecursiveCharacterTextSplitter(chunk_size=600,
                                               chunk_overlap=200)

# Cắt các document thành các đoạn nhỏ và gán tên file vào metadata
chunked_documents = []
for doc in documents:
    # Tạo các đoạn nhỏ từ từng document
    chunks = text_splitter.split_documents([doc])
    # Thêm metadata cho mỗi chunk chỉ chứa tên file (không có đường dẫn)
    for chunk in chunks:
        file_name = os.path.basename(doc.metadata.get('source', 'unknown'))  # Lấy tên file từ đường dẫn
        chunk.metadata['source'] = file_name  # Gán tên file vào metadata
        chunked_documents.append(chunk)

In [11]:
len(chunked_documents)

1602

In [12]:
# Kiểm tra nội dung của các đoạn văn bản
for i, doc in enumerate(chunked_documents):
    print(f"Document {i} content: {doc.page_content[:100]}")  # In 100 ký tự đầu tiên
    print(f"Document {i} metadata: {doc.metadata}")
    if not doc.page_content.strip():
        print(f"Document {i} is empty!")

Document 0 content: Ngành Trí tuệ nhân tạo
Chương trình: Đại học Chính quy
Khóa tuyển: 2023
(Ban hành kèm theo Quyết địn
Document 0 metadata: {'source': 'TTNT.txt'}
Document 1 content: 1. Thông tin chung về chương trình đào tạo
1.1. Tên ngành: 
- Tiếng Việt: Trí tuệ nhân tạo
- Tiếng A
Document 1 metadata: {'source': 'TTNT.txt'}
Document 2 content: - Tiếng Anh Bachelor of Science in Artificial Intelligence 
1.8. Ngôn ngữ giảng dạy tiếng Việt
1.9. 
Document 2 metadata: {'source': 'TTNT.txt'}
Document 3 content: 2. Mục tiêu đào tạo
2.1. Mục tiêu chung
Mục tiêu của chương trình là nhằm đào tạo ra các sinh viên t
Document 3 metadata: {'source': 'TTNT.txt'}
Document 4 content: - Được trang bị những kỹ năng cá nhân, kỹ năng nhóm, kỹ năng giao tiếp và kỹ năng phân tích để nhận 
Document 4 metadata: {'source': 'TTNT.txt'}
Document 5 content: 2.2. Mục tiêu cụ thể
PO1: Nhận biết được trách nhiệm, đạo đức nghề nghiệp, và hiện trạng kinh tế, mô
Document 5 metadata: {'source': 'TTNT.txt'}
Document 6

In [13]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

# 1. Sử dụng chunked_documents từ TextSplitter
print(f"Số lượng chunks để add vào Qdrant: {len(chunked_documents)}")

# 2. Tạo embeddings và add vào Qdrant
if len(chunked_documents) > 0:
    # Khởi tạo embeddings
    embeddings = HuggingFaceInferenceAPIEmbeddings(
        model_name=EMBEDDINGS_MODEL_NAME,
        api_key=HUGGINGFACE_API_KEY
    )

    # Tạo vector database từ chunks
    qdrant = Qdrant.from_documents(
        chunked_documents,
        embeddings,
        url=QDRANT_URL,
        prefer_grpc=False,
        collection_name=COLLECTION_NAME,  # Đổi tên collection để phân biệt
        api_key=QDRANT_API_KEY
    )
    print("Đã tạo vector database thành công!")
else:
    print("Không có dữ liệu chunks để tạo vector database!")

Số lượng chunks để add vào Qdrant: 1602
Đã tạo vector database thành công!


In [14]:
# Test tìm kiếm
query = "Các môn học Học kỳ 2 năm 3 ngành hệ thống thông tin"
found_docs = qdrant.similarity_search(query, k=10)

print("\nKết quả test tìm kiếm:")
for i, doc in enumerate(found_docs):
    print(f"\nKết quả {i+1}:")
    print(doc.page_content)
    print(doc.metadata)
    print("-" * 80)


Kết quả test tìm kiếm:

Kết quả 1:
8.2.6. Các môn học Học kỳ 2 năm 3 Chuyên ngành Công nghệ thông tin
BAA00103 Chủ nghĩa xã hội khoa học
CSC10107 Thực tập thực tế
CSC11006 Nhập môn điện toán đám mây
CSC15003 Mã hóa ứng dụng
CSC10102 Kiến tập nghề nghiệp
CSC10103 Khởi nghiệp
CSC10104 Quy hoạch tuyển tinh
CSC10106 Thuật toán tổ hợp và ứng dụng
CSC12004 Phân tích thiết kế hệ thống thông tin
CSC11006 Nhập môn điện toán đám mây
CSC12105 Thương mại điện tử
CSC13010 Thiết kế phần mềm
CSC14005 Nhập môn học máy
CSC14118 Nhập môn dữ liệu lớn
CSC17001 Phân tích dữ liệu thông minh
CSC15004 Học thống kê
{'source': 'CNTT.txt', '_id': '867dc2de-032d-4ff1-8392-531220a1cb79', '_collection_name': 'ITUS_e5_R_600'}
--------------------------------------------------------------------------------

Kết quả 2:
8.1.6. Các môn học Học kỳ 2 năm 3 Chuyên ngành Mạng máy tính và Viễn thông
BAA00103 Chủ nghĩa xã hội khoa học
CSC10107 Thực tập thực tế (Học kỳ hè năm 3)
CSC11002 Hệ thống viễn thông
CSC11003 Lập trình

# **Sử dụng TextSplitter cải tiến Chunking theo đề mục**

In [20]:
import re
from typing import List
from langchain.docstore.document import Document
from langchain.text_splitter import TextSplitter

class TextSplitter(TextSplitter):
    def __init__(self, chunk_size: int = 600, chunk_overlap: int = 200):
        super().__init__(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        print(f"Initialized with chunk_size={chunk_size}, chunk_overlap={chunk_overlap}")

    def is_title(self, line: str) -> bool:
        return bool(re.match(r'^\d+(\.\d+)*\.', line.strip()))

    def find_last_title(self, lines: List[str]) -> str:
        """Tìm dòng tiêu đề cuối cùng trong danh sách dòng."""
        for line in reversed(lines):
            if self.is_title(line):
                return line
        return ""

    def normalize_whitespace(self, text: str) -> str:
        """Đảm bảo nội dung không chứa nhiều hơn 2 ký tự xuống dòng liên tiếp."""
        return re.sub(r'(\n{2,})', '\n\n', text)

    def find_nearest_split_point(self, text: str) -> int:
        """Tìm vị trí khoảng trắng gần nhất để cắt."""
        split_point = max(text.rfind(' '), text.rfind('!'), text.rfind('?'))
        return split_point if split_point != -1 else len(text)

    def split_text(self, text: str) -> List[str]:
        print("\nStarting split_text...")
        text = self.normalize_whitespace(text)  # Chuẩn hóa các ký tự xuống dòng
        chunks = []
        current_chunk = []
        current_length = 0
        lines = text.splitlines()
        last_title = ""

        for line in lines:
            line_length = len(line)

            if current_length + line_length > self.chunk_size:
                # Đảm bảo kích thước chunk không vượt quá chunk_size
                chunk = "\n".join(current_chunk).strip()
                if len(chunk) > self.chunk_size:
                    split_point = self.find_nearest_split_point(chunk[:self.chunk_size])
                    chunks.append(chunk[:split_point + 1].strip())
                    overlap_text = chunk[split_point + 1:]
                else:
                    chunks.append(chunk)
                    overlap_text = "\n".join(current_chunk)[-self.chunk_overlap:]

                overlap_lines = overlap_text.splitlines()

                # Kiểm tra nếu overlap chứa dòng tiêu đề
                contains_title = any(self.is_title(ol) for ol in overlap_lines)

                if contains_title:
                    # Cắt bỏ nội dung trước dòng tiêu đề đầu tiên trong overlap
                    for i, ol in enumerate(overlap_lines):
                        if self.is_title(ol):
                            overlap_lines = overlap_lines[i:]
                            break
                elif last_title:
                    # Nếu dòng đầu tiên của overlap không phải tiêu đề, cắt bỏ dòng đầu tiên
                    if overlap_lines and not self.is_title(overlap_lines[0]):
                        overlap_lines.pop(0)

                    # Thêm dòng tiêu đề cuối cùng từ chunk trước đó
                    overlap_lines.insert(0, last_title)

                overlap = "\n".join(overlap_lines).strip()

                # Đảm bảo dòng tiêu đề có thể được sử dụng nhiều lần
                current_chunk = overlap_lines.copy() if overlap else []
                current_length = len(overlap)

                last_title = self.find_last_title(current_chunk)  # Cập nhật dòng tiêu đề cuối cùng

            current_chunk.append(line)
            current_length += line_length + 1

            if self.is_title(line):
                last_title = line  # Cập nhật dòng tiêu đề nếu tìm thấy

        if current_chunk:
            chunk = "\n".join(current_chunk).strip()
            if len(chunk) < self.chunk_overlap and len(chunks) > 0:
                chunks[-1] += "\n" + chunk
            elif len(chunk) > self.chunk_size:
                split_point = self.find_nearest_split_point(chunk[:self.chunk_size])
                chunks.append(chunk[:split_point + 1].strip())
                chunks.append(chunk[split_point + 1:].strip())
            else:
                chunks.append(chunk)

        print(f"\nTotal chunks created: {len(chunks)}")
        return chunks

    def create_documents(self, texts: List[str], metadatas: List[dict] = None) -> List[Document]:
        print("\nStarting create_documents...")
        documents = []
        for i, text in enumerate(texts):
            print(f"\nProcessing text {i+1}/{len(texts)}")
            metadata = metadatas[i] if metadatas else {}
            chunks = self.split_text(text)
            for j, chunk in enumerate(chunks):
                doc = Document(page_content=chunk, metadata=metadata.copy())
                documents.append(doc)
                print(f"Created document {j+1}/{len(chunks)} for text {i+1}")
        print(f"\nTotal documents created: {len(documents)}")
        return documents


In [21]:
chunk_size = 600
chunk_overlap = 200
# 1. Khởi tạo splitter
text_splitter = TextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

# 2. Tạo danh sách chứa tất cả các chunks từ nhiều tài liệu
chunked_documents = []
for doc in documents:
    # Lấy tên file từ metadata source
    filename = os.path.basename(doc.metadata['source']) if 'source' in doc.metadata else "unknown"

    # Tạo các chunks với metadata là tên file
    chunks = text_splitter.create_documents(
        texts=[doc.page_content],
        metadatas=[{"source": filename}]
    )
    chunked_documents.extend(chunks)

# 3. Thống kê kết quả
print(f"Số lượng chunks được tạo: {len(chunked_documents)}\n")

# 4. In mẫu một số chunks (tối đa 5 để tránh quá dài)
for i, doc in enumerate(chunked_documents[:5]):
    print("============================")
    print(f"Chunk {i+1}:")
    print(doc.page_content)
    print(f"Metadata: {doc.metadata}")
    print("-" * 80)

# 5. Tính toán độ dài trung bình
lengths = [len(doc.page_content) for doc in chunked_documents]
if lengths:  # Tránh lỗi chia cho 0 nếu không có chunk nào
    avg_length = sum(lengths) / len(lengths)
    print(f"\nĐộ dài trung bình của chunks: {avg_length:.2f} ký tự")
    print(f"Chunk ngắn nhất: {min(lengths)} ký tự")
    print(f"Chunk dài nhất: {max(lengths)} ký tự")
else:
    print("\nKhông có chunks nào được tạo.")


Initialized with chunk_size=600, chunk_overlap=200

Starting create_documents...

Processing text 1/1

Starting split_text...

Total chunks created: 68
Created document 1/68 for text 1
Created document 2/68 for text 1
Created document 3/68 for text 1
Created document 4/68 for text 1
Created document 5/68 for text 1
Created document 6/68 for text 1
Created document 7/68 for text 1
Created document 8/68 for text 1
Created document 9/68 for text 1
Created document 10/68 for text 1
Created document 11/68 for text 1
Created document 12/68 for text 1
Created document 13/68 for text 1
Created document 14/68 for text 1
Created document 15/68 for text 1
Created document 16/68 for text 1
Created document 17/68 for text 1
Created document 18/68 for text 1
Created document 19/68 for text 1
Created document 20/68 for text 1
Created document 21/68 for text 1
Created document 22/68 for text 1
Created document 23/68 for text 1
Created document 24/68 for text 1
Created document 25/68 for text 1
Created

In [22]:
COLLECTION_NAME = "ITUS_e5_600v2"

In [23]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

# 1. Sử dụng chunked_documents từ TextSplitter
print(f"Số lượng chunks để add vào Qdrant: {len(chunked_documents)}")

# 2. Tạo embeddings và add vào Qdrant
if len(chunked_documents) > 0:
    # Khởi tạo embeddings
    embeddings = HuggingFaceInferenceAPIEmbeddings(
        model_name=EMBEDDINGS_MODEL_NAME,
        api_key=HUGGINGFACE_API_KEY
    )

    # Tạo vector database từ chunks
    qdrant = Qdrant.from_documents(
        chunked_documents,
        embeddings,
        url=QDRANT_URL,
        prefer_grpc=False,
        collection_name=COLLECTION_NAME,  # Đổi tên collection để phân biệt
        api_key=QDRANT_API_KEY
    )
    print("Đã tạo vector database thành công!")
else:
    print("Không có dữ liệu chunks để tạo vector database!")

Số lượng chunks để add vào Qdrant: 1539
Đã tạo vector database thành công!


In [24]:
# Test tìm kiếm
query = "Các môn học Học kỳ 2 năm 3 ngành hệ thống thông tin"
found_docs = qdrant.similarity_search(query, k=10)

print("\nKết quả test tìm kiếm:")
for i, doc in enumerate(found_docs):
    print(f"\nKết quả {i+1}:")
    print(doc.page_content)
    print(doc.metadata)
    print("-" * 80)


Kết quả test tìm kiếm:

Kết quả 1:
8.2.6. Các môn học Học kỳ 2 năm 3 Chuyên ngành Công nghệ thông tin
BAA00103 Chủ nghĩa xã hội khoa học
CSC10107 Thực tập thực tế
CSC11006 Nhập môn điện toán đám mây
CSC15003 Mã hóa ứng dụng
CSC10102 Kiến tập nghề nghiệp
CSC10103 Khởi nghiệp
CSC10104 Quy hoạch tuyển tinh
CSC10106 Thuật toán tổ hợp và ứng dụng
CSC12004 Phân tích thiết kế hệ thống thông tin
CSC11006 Nhập môn điện toán đám mây
CSC12105 Thương mại điện tử
CSC13010 Thiết kế phần mềm
CSC14005 Nhập môn học máy
CSC14118 Nhập môn dữ liệu lớn
CSC17001 Phân tích dữ liệu thông minh
CSC15004 Học thống kê
CSC15007 Thống kê máy tính và ứng
{'source': 'CNTT.txt', '_id': '983b2ee8-c625-43e8-90f2-3b64f20b7777', '_collection_name': 'ITUS_e5_600v2'}
--------------------------------------------------------------------------------

Kết quả 2:
8.2.6. Các môn học Học kỳ 2 năm 3 Chuyên ngành Công nghệ thông tin
CSC15107 Phân tích dữ liệu bảo toàn

8.2.7. Các môn học Học kỳ 1 năm 4 Chuyên ngành Công nghệ thông 